# Web Scraping for Indeed.com & Predicting Salaries

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary predictor with Logistic Regression.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use Logistic Regression.

- Question: Why would we want this to be a classification problem?
- Answer: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

In [12]:
URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

In [13]:
import requests
import bs4
from bs4 import BeautifulSoup

In [14]:
## YOUR CODE HERE
r = requests.get(URL)
soup = BeautifulSoup(r.content, 'lxml')

In [15]:
print soup.prettify()


<!DOCTYPE html>
<html lang="en">
 <head>
  <meta content="text/html;charset=utf-8" http-equiv="content-type"/>
  <!-- pll -->
  <script src="/s/4a89ac5/en_US.js" type="text/javascript">
  </script>
  <link href="/s/a6a334e/jobsearch_all.css" rel="stylesheet" type="text/css"/>
  <link href="http://rss.indeed.com/rss?q=data+scientist+%2420%2C000&amp;l=New+York" rel="alternate" title="Data Scientist $20,000 Jobs, Employment in New York State" type="application/rss+xml"/>
  <link href="/m/jobs?q=data+scientist+%2420%2C000&amp;l=New+York" media="handheld" rel="alternate"/>
  <script type="text/javascript">
   window['closureReadyCallbacks'] = [];

    function call_when_jsall_loaded(cb) {
        if (window['closureReady']) {
            cb();
        } else {
            window['closureReadyCallbacks'].push(cb);
        }
    }
  </script>
  <script src="/s/40c36c1/jobsearch-all-compiled.js" type="text/javascript">
  </script>
  <script type="text/javascript">
   var pingUrlsForGA = [];

v

In [23]:
results = soup.findAll('div', {'class' :' row  result'})
results

[<div class=" row result" data-jk="5b8a65b6982893bf" data-tn-component="organicJob" id="p_5b8a65b6982893bf" itemscope="" itemtype="http://schema.org/JobPosting">\n<h2 class="jobtitle" id="jl_5b8a65b6982893bf">\n<a class="turnstileLink" data-tn-element="jobTitle" href="/rc/clk?jk=5b8a65b6982893bf&amp;fccid=6ae0e4a8a0a337cd" itemprop="title" onclick="setRefineByCookie(['salest']); return rclk(this,jobmap[0],true,0);" onmousedown="return rclk(this,jobmap[0],0);" rel="nofollow" target="_blank" title="Data Scientist"><b>Data</b> <b>Scientist</b></a>\n</h2>\n<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">\n<span itemprop="name">\n<a href="/cmp/Placeiq?from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=5b8a65b6982893bf&amp;jcid=6ae0e4a8a0a337cd" target="_blank">\n    PlaceIQ</a></span>\n</span>\n\n - <a class="turnstileLink slNoUnderline " data-tn-element="reviewStars" data-tn-variant="cmplinktst2" href="/cmp/Placeiq/reviews?jcid=6ae0e4a8a0

Let's look at one result more closely. A single `result` looks like

```
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=2480d203f7e97210&amp;jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a `nobr` element inside of a `td` element with `class='snip`.
- The title of a job is in a link with class set to `jobtitle` and a `data-tn-element="jobTitle`.  
- The location is set in a `span` with `class='location'`. 
- The company is set in a `span` with `class='company'`. 

### Write 4 functions to extract each item: location, company, job, and salary.

example: 
```python
def extract_location_from_result(result):
    return result.find ...
```


- Make sure these functions are robust and can handle cases where the data/field may not be available.
- Test the functions on the results above

In [21]:
## YOUR CODE HERE
def extract_text(el):
    if el:
        return el.text.strip()
    else:
        return ''

def get_company(result):
    return extract_text(result.find('span', {'class':'company'}))

In [25]:
company = soup.select('span.company')
company[3].text.strip()

u'Google'

Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).

#### Complete the following code to collect results from multiple cities and starting points. 
- Enter your city below to add it to the search
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

In [5]:
YOUR_CITY = ''

In [6]:
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 100

results = []

for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Los+Angeles%2C+CA']):
    for start in range(0, max_results_per_city, 10):
        # Grab the results from the request (as above)
        # Append to the full set of results
        pass

#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [7]:
## YOUR CODE HERE

Lastly, we need to clean up salary data. 
1. Some of the salaries are not yearly but hourly, these will be useful to us for now
2. The salaries are given as text and usually with ranges.

#### Filter out the salaries that are not yearly (filter those that refer to hour)

In [8]:
## YOUR CODE HERE

#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

In [9]:
## YOUR CODE HERE

### Save your results as a CSV

## Predicting salaries using Logistic Regression

#### Load in the the data of scraped salaries

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from patsy import dmatrix
from sklearn.cross_validation import cross_val_score

In [2]:
df = pd.read_csv('/Users/nfergie/Desktop/DSI_SM_01/projects/01-projects-weekly/project-04/assets/indeed-scraped-job-postings.csv')
df.head()

,city,company,salary,summary,title,parsed_salary
0,San+Francisco,MarkMonitor,"$180,000 a year","Data skills (SQL, Hive, Pig). Applying machine...",Data Scientist,180000.0
1,San+Francisco,Workbridge Associates,"$130,000 - $180,000 a year",3+ years of industry experience in a data scie...,Senior Data Scientist,155000.0
2,San+Francisco,Mines.io,"$80,000 - $120,000 a year",We are looking for a data scientist/developer ...,Full-Stack Data Scientist,100000.0
3,San+Francisco,Workbridge Associates,"$150,000 - $180,000 a year",In this position you will share programming an...,Data Scientist,165000.0
4,San+Francisco,Smith Hanley Associates,"$140,000 a year","This person will recruit, build and lead a tea...",Data Scientist,140000.0


#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

In [3]:
meansal = np.mean(df['parsed_salary'])
df['bin_sal'] = [0 if i < meansal else 1 for i in df['parsed_salary']]
df.head()

,city,company,salary,summary,title,parsed_salary,bin_sal
0,San+Francisco,MarkMonitor,"$180,000 a year","Data skills (SQL, Hive, Pig). Applying machine...",Data Scientist,180000.0,1
1,San+Francisco,Workbridge Associates,"$130,000 - $180,000 a year",3+ years of industry experience in a data scie...,Senior Data Scientist,155000.0,1
2,San+Francisco,Mines.io,"$80,000 - $120,000 a year",We are looking for a data scientist/developer ...,Full-Stack Data Scientist,100000.0,1
3,San+Francisco,Workbridge Associates,"$150,000 - $180,000 a year",In this position you will share programming an...,Data Scientist,165000.0,1
4,San+Francisco,Smith Hanley Associates,"$140,000 a year","This person will recruit, build and lead a tea...",Data Scientist,140000.0,1


#### Thought experiment: What is the baseline accuracy for this model?

For a random guess the accuracy should be .50 so we aim for something above that in our modelling

#### Create a Logistic Regression model to predict High/Low salary using statsmodel. Start by ONLY using the location as a feature. Display the coefficients and write a short summary of what they mean.

In [4]:
model = sm.logit("bin_sal ~ city", data=df).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.272087
         Iterations 11


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                bin_sal   No. Observations:                 4791
Model:                          Logit   Df Residuals:                     4786
Method:                           MLE   Df Model:                            4
Date:                Wed, 27 Jul 2016   Pseudo R-squ.:                  0.6075
Time:                        18:31:46   Log-Likelihood:                -1303.6
converged:                       True   LL-Null:                       -3320.8
                                        LLR p-value:                     0.000
=========================================================================================
                            coef    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------------
Intercept                -1.7557      0.054    -32.220      0.000        -1.862    -1.649
city[T.Chicago]           8.8238      1.002      8.807      0.000         6.860    10.788
city[T.New+York]          4.6107      0.217     21.260      0.000         4.186     5.036
city[T.San+Francisco]     5.7962      0.385     15.049      0.000         5.041     6.551
city[T.Seattle]           0.4516      0.264      1.708      0.088        -0.067     0.970
=========================================================================================
"""

All of our p-values here are quite small except for that of seattle. the CI for the variables are also all relatively small, with chicago having the largest one. Also, the Pseudo R-squared value is only at .6075 which is smaller than i'd like it to be

#### Create a few new variables in your dataframe to represent interesting features of a job title.
- For example, create a feature that represents whether 'Senior' is in the title 
- or whether 'Manager' is in the title. 
- Then build a new Logistic Regression model with these features. Do they add any value? 


In [5]:
df['senior'] = df['title'].str.contains('Senior').astype(int)
df['director'] = df['title'].str.contains('Director').astype(int)
df['manager'] = df['title'].str.contains('Manager').astype(int)
df.head()


,city,company,salary,summary,title,parsed_salary,bin_sal,senior,director,manager
0,San+Francisco,MarkMonitor,"$180,000 a year","Data skills (SQL, Hive, Pig). Applying machine...",Data Scientist,180000.0,1,0,0,0
1,San+Francisco,Workbridge Associates,"$130,000 - $180,000 a year",3+ years of industry experience in a data scie...,Senior Data Scientist,155000.0,1,1,0,0
2,San+Francisco,Mines.io,"$80,000 - $120,000 a year",We are looking for a data scientist/developer ...,Full-Stack Data Scientist,100000.0,1,0,0,0
3,San+Francisco,Workbridge Associates,"$150,000 - $180,000 a year",In this position you will share programming an...,Data Scientist,165000.0,1,0,0,0
4,San+Francisco,Smith Hanley Associates,"$140,000 a year","This person will recruit, build and lead a tea...",Data Scientist,140000.0,1,0,0,0


In [6]:
model = sm.logit("bin_sal ~ city + senior + manager", data=df).fit()
model.summary()


         Current function value: 0.263784
         Iterations: 35


//anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                bin_sal   No. Observations:                 4791
Model:                          Logit   Df Residuals:                     4784
Method:                           MLE   Df Model:                            6
Date:                Wed, 27 Jul 2016   Pseudo R-squ.:                  0.6194
Time:                        18:32:01   Log-Likelihood:                -1263.8
converged:                      False   LL-Null:                       -3320.8
                                        LLR p-value:                     0.000
=========================================================================================
                            coef    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------------
Intercept                -1.6075      0.056    -28.747      0.000        -1.717    -1.498
city[T.Chicago]           9.4313      1.023      9.217      0.000         7.426    11.437
city[T.New+York]          4.5668      0.219     20.813      0.000         4.137     4.997
city[T.San+Francisco]     7.3002      0.456     16.015      0.000         6.407     8.194
city[T.Seattle]           0.1660      0.283      0.586      0.558        -0.389     0.721
senior                   -1.7338      0.258     -6.711      0.000        -2.240    -1.227
manager                  26.9784   2.01e+05      0.000      1.000     -3.94e+05  3.94e+05
=========================================================================================
"""

#### Rebuild this model with scikit-learn.
- You can either create the dummy features manually or use the `dmatrix` function from `patsy`
- Remember to scale the feature variables as well!


In [13]:
scaler = StandardScaler()
model = LogisticRegression()

dum_x = dmatrix("city + senior + manager + 0", data=df) 
X = scaler.fit_transform(dum_x)
y = df['bin_sal']

model.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

#### Use cross-validation in scikit-learn to evaluate the model above. 
- Evaluate the accuracy, AUC, precision and recall of the model. 
- Discuss the differences and explain when you want a high-recall or a high-precision model in this scenario.

In [14]:
scores1 = cross_val_score(model, X, y, cv=3, scoring='accuracy')
scores2 = cross_val_score(model, X, y, cv = 3, scoring = 'precision')
scores3 = cross_val_score(model, X, y, cv = 3, scoring= 'roc_auc')
scores4 = cross_val_score(model, X, y, cv = 3, scoring = 'recall')
print('accuracy', scores1.mean(), scores1.std())
print('precision', scores2.mean(), scores2.std())
print('roc_auc', scores3.mean(), scores3.std())
print('recall', scores4.mean(), scores4.std())

('accuracy', 0.81569609684825706, 0.098970285405032721)
('precision', 0.95063710499490328, 0.0354208192383869)
('roc_auc', 0.76473216059490134, 0.15124417334164142)
('recall', 0.674989596337911, 0.23106212628201972)


#### Compare L1 and L2 regularization for this logistic regression model. What effect does this have on the coefficients learned?

In [18]:
model = LogisticRegression(penalty= 'l1')
for i in ['accuracy', 'precision', 'roc_auc', 'recall']:
    score = cross_val_score(model, X, y, cv = 3, scoring = i)
    print (i, score.mean(), score.std())

('accuracy', 0.89229805886036306, 0.10763760559472856)
('precision', 0.95063710499490328, 0.0354208192383869)
('roc_auc', 0.8849339289037782, 0.12312976307706712)
('recall', 0.82771535580524347, 0.24364728040884781)


In [19]:
model = LogisticRegression(penalty= 'l2')
for i in ['accuracy', 'precision', 'roc_auc', 'recall']:
    score = cross_val_score(model, X, y, cv = 3, scoring = i)
    print (i, score.mean(), score.std())

('accuracy', 0.81569609684825706, 0.098970285405032721)
('precision', 0.95063710499490328, 0.0354208192383869)
('roc_auc', 0.76473216059490134, 0.15124417334164142)
('recall', 0.674989596337911, 0.23106212628201972)


#### Continue to incorporate other text features from the title or summary that you believe will predict the salary and examine their coefficients

#### Take ~100 scraped entries with salaries. Convert them to use with your model and predict the salary - which entries have the highest predicted salaries?

### BONUS 

#### Bonus: Use Count Vectorizer from scikit-learn to create features from the text summaries. 
- Examine using count or binary features in the model
- Re-evaluate the logistic regression model using these. Does this improve the model performance? 
- What text features are the most valuable? 

In [23]:
## YOUR CODE HERE

In [24]:
## YOUR CODE HERE

#### Re-test L1 and L2 regularization. You can use LogisticRegressionCV to find the optimal reguarlization parameters. 
- Re-test what text features are most valuable.  
- How do L1 and L2 change the coefficients?

In [25]:
## YOUR CODE HERE